In [1]:
import pandas as pd
import os

# os.environ['PYSPARK_SUBMIT_ARGS'] = "--master mymaster --total-executor 2 --conf 'spark.driver.extraJavaOptions=-Dhttp.proxyHost=proxy.mycorp.com-Dhttp.proxyPort=1234' -Dhttp.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 -Dhttps.proxyHost=proxy.mycorp.com -Dhttps.proxyPort=1234 -Dhttps.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 pyspark-shell"
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, isnan, when, count
from pyspark.ml.feature import Imputer, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import os

from DataLoader import DataLoader
from DataSetEnum import DataSet


In [2]:
import matplotlib.pyplot as plt


In [3]:
loader = DataLoader(data_set=DataSet.TRAIN)
df = loader.df

24/03/13 11:33:48 WARN Utils: Your hostname, Juliuss-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 172.18.14.183 instead (on interface en0)
24/03/13 11:33:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/13 11:33:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/13 11:33:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=22680Kb max_used=22684Kb free=108391Kb
 bounds [0x00000001099e0000, 0x000000010b030000, 0x00000001119e0000]
 total_blobs=8965 nmethods=8035 adapters=842
 compilation: disabled (not enough contiguous free space left)
Removed 0 duplicate rows.


In [4]:
df.columns

['',
 'product_id',
 'product_parent',
 'product_title',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date',
 'marketplace_id',
 'product_category_id',
 'label']

In [5]:
unique = ["", "review_headline", "review_body"]

In [6]:
columns_to_visualize = [col for col in df.columns if col not in unique]

In [7]:
# taking sample of 0.8 of whole data
# convert it to pandas dataframe 
sampled_data = df\
    .sample(withReplacement=False, fraction=0.1, seed=42)\
    .toPandas()
# and at the end lets use our beautiful matplotlib
# plt.scatter(sampled_data.x,sampled_data.y)
# plt.xlabel('x')
# plt.ylabel('y')
# plt.title('relation of y and x')
# plt.show()

In [8]:
sampled_data.head(5)

,,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
0,56,B00AQL8VU4,496201564,Amazon for Tablets,N,Y,Five Stars,Excellent,2014-10-26,1,1,False
1,139,B0000AGDMJ,579654184,Das Leben des David Gale,N,N,"Dás Bésté, wás ich séit lángém géséhén hábé",Dér Film hátté schớn éiné gúté Béschréibúng in...,2007-12-31,0,11,True
2,170,B0000252BG,514416658,Aenima,N,N,anything Keenan touches turns to gold,It's perhaps not as advanced as Lateralus but ...,2001-07-21,1,16,False
3,469,B008VQVK2W,81198786,"Castle, saison 4 - coffret 6 DVD",N,Y,bonne série,"a voir et a revoir , cette série est pétillant...",2014-01-19,2,3,False
4,545,0747587906,613676928,Restless,N,Y,Five Stars,excellent,2015-04-28,1,0,False


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_data_distribution(df, ignore: list):
    """
    Visualizes the data distribution for each column in a DataFrame.
    Uses histograms/density plots for numeric columns and bar plots for categorical columns.
    """
    # Set the aesthetics for seaborn plots
    sns.set(style='whitegrid')

    # Determine the number of rows needed in the subplot grid
    n_cols = df.shape[1]
    n_rows = n_cols // 2 if n_cols % 2 == 0 else (n_cols // 2) + 1

    # Create a figure with subplots
    fig, axes = plt.subplots(n_rows, 2, figsize=(14, n_rows * 4))
    axes = axes.ravel()  # Flatten the array of axes

    for i, column in enumerate([col for col in df.columns if col not in ignore]):
        ax = axes[i]
        
        # Check if the column is numeric or categorical
        if pd.api.types.is_numeric_dtype(df[column]):
            # For numeric data, use a histogram
            sns.histplot(df[column], kde=True, ax=ax)
            ax.set_title(f'Distribution of {column} (Numeric)')
            ax.set_xlabel(column)
            ax.set_ylabel('Frequency')
        else:
            # For categorical data, use a bar plot of value counts
            value_counts = df[column].value_counts()
            sns.barplot(x=value_counts.index, y=value_counts.values, ax=ax)
            ax.set_title(f'Distribution of {column} (Categorical)')
            ax.set_xlabel(column)
            ax.set_ylabel('Count')
            plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

        ax.grid(True)  # Add grid for easier reading

    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()

# Example usage:
# df = pd.DataFrame(your_data)
# visualize_data_distribution(df)


In [10]:
visualize_data_distribution(sampled_data)

/var/folders/pz/pqdfb44x6z56nqs5vr77b6rw0000gn/T/ipykernel_3988/2268280278.py:43: UserWarning: Glyph 26 () missing from current font.
  plt.tight_layout()
/var/folders/pz/pqdfb44x6z56nqs5vr77b6rw0000gn/T/ipykernel_3988/2268280278.py:43: UserWarning: Glyph 128544 (\N{ANGRY FACE}) missing from current font.
  plt.tight_layout()
/var/folders/pz/pqdfb44x6z56nqs5vr77b6rw0000gn/T/ipykernel_3988/2268280278.py:43: UserWarning: Glyph 127775 (\N{GLOWING STAR}) missing from current font.
  plt.tight_layout()
/var/folders/pz/pqdfb44x6z56nqs5vr77b6rw0000gn/T/ipykernel_3988/2268280278.py:43: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()
/Users/julius/repos/BD-Project/.venv/lib/python3.12/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 26 () missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/Users/julius/repos/BD-Project/.venv/lib/python3.12/site-packages/IPy